In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# torch.set_default_dtype(torch.float32)
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
    torch.set_default_device(device)

Running on the GPU


In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

> 这是自定义的数据集

In [4]:
train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=128, shuffle=True, generator=torch.Generator(device), num_workers=8)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True, generator=torch.Generator(device), num_workers=8)

In [5]:
for x, y in train_dataloader:
    print(x, y)
    print(x.dtype)
    break;

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..

> Q: 如何以float32形式读取数据?
> A: 自定义数据集getItem函数返回np.array dtype=np.float32

In [12]:

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 10)
).cuda()
# def init_weights 这里省略, 因为默认会init的

In [13]:
lr = 0.001
max_epochs = 20
lambd = 0.01

In [14]:
loss = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, weight_decay=0.02, betas=(0.9, 0.99))

In [15]:
history = []
for epoch in range(max_epochs):
    l = None
    for x, y in train_dataloader:
        l = loss(model(x.to(device)), y.to(device))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        print(f"\r epoch {epoch} loss = {l}", end='')
    history.append(l.cpu().detach().numpy())
    print("")

 epoch 0 loss = 0.66034901142120367
 epoch 1 loss = 0.70630949735641483
 epoch 2 loss = 0.54352074861526493
 epoch 3 loss = 0.78621166944503785
 epoch 4 loss = 0.54051274061203187
 epoch 5 loss = 0.52439886331558237
 epoch 6 loss = 0.65418583154678344
 epoch 7 loss = 0.63215011358261115
 epoch 8 loss = 0.59643036127090456
 epoch 9 loss = 0.71630209684371954
 epoch 10 loss = 0.71540600061416634
 epoch 11 loss = 0.64970773458480834
 epoch 12 loss = 0.75852149724960333
 epoch 13 loss = 0.63635736703872684
 epoch 14 loss = 0.48871478438377385
 epoch 15 loss = 0.55355161428451547
 epoch 16 loss = 0.50898075103759776
 epoch 17 loss = 0.52587383985519416
 epoch 18 loss = 0.54543846845626834
 epoch 19 loss = 0.51790100336074837


In [ ]:
len(test_data)

In [17]:
true_flag = 0
with torch.no_grad():
    for x, y in test_dataloader:
        result = torch.argmax(model(x.to(device)), 1).cpu()
        true_flag += torch.where(result == y, 1, 0).sum()
true_flag

tensor(7950)

In [18]:
acc = true_flag / len(test_data)
acc

tensor(0.7950)